# FINANCIAL MODELLING SIMULATION

# This is a group project done by the students in Group C in CSC 408 (Modelling and Simulation) Under the supervision of Dr. Awoyemi


The numpy_financial package contains this Mathematical Formula 

A = P(i + i/((1+i)**n - 1))

A - Periodic Payment amount
P = Amount of principal
i = Periodic Interest
n = Total number of payments

In [2]:
# Libraries needed 
import numpy as np
import numpy_financial as npf
import pandas as pd
from IPython.display import display, HTML

In [3]:
#Input Validation for Principal, annual rate , loan term and no of payments
class InputValidation:
    """
    This class validates the inputs (Principal, annual rate, term in years and number of payments per year)
    Principal(principal) : Validates if input Loan amount is a positive integer
    Annual Rate(annual_interest_rate) : Validates if the Annual rate is >0
    Term in Years(per): Validates if term is a positive finite integer
    No of Payments(nper):Validates if nper is a positive finite integer
    """
    def __init__(self,inputvar):
        self.inputvar = inputvar
    
    def age(self):
        while True:
            try:
                tempvar = int(input(f"\nPlease Enter a positive integer value for {self.inputvar}:"))
                if tempvar.is_integer() and tempvar>18 and tempvar <= 25:
                    print("You can't borrow more than 2000 dollars")
                    break 
                else:
                    print(f"\n{self.inputvar} should be a positive integer value!")
            except:
                print(f"\n{self.inputvar} is not a number!")
    
        return tempvar

    
    #Method for checking if the input variable is a positive integer value (Eg. Principal)
    def check_positive_integer(self):
        while True:
            try:
                tempvar = float(input(f"\nPlease Enter a positive integer value for {self.inputvar}:"))
                if tempvar.is_integer() and tempvar>0:
                    break 
                else:
                    print(f"\n{self.inputvar} should be a positive integer value!")
            except:
                print(f"\n{self.inputvar} is not a number!")
    
        return tempvar

    #Method for checking if the input variable is a positive value (Eg. Annual interest rate)
    def check_positive_float(self):
        while True:
            try:
                tempvar = float(input(f"\nPlease Enter a positive integer value for {self.inputvar}:"))
                if tempvar>0 and tempvar < 100:
                    break
                else:
                    print(f"\n{self.inputvar} should be a positive value greater than 0 and less than 100!")
            except:
                print("\nNot a valid Option!! Try again..")
        return tempvar
        
    #Method for checking if the input variable is a finite term (Eg.Loan term and No. of payments per year)
    def finiteterm_check(self):
        while True:
            try:
                tempvar = float(input(f"\nPlease Enter a positive integer value for {self.inputvar}:"))
                if tempvar.is_integer() and tempvar>0 and tempvar<100:
                    break 
                else:
                    print(f"\n{self.inputvar} should be a positive integer value less than 100!")
            except:
                print(f"\n{self.inputvar} is not a number!")
    
        return tempvar
       

In [4]:
# Function to generate matrix of various term vs rate of interest
def generate_matrix(principal,paymentfreq,initial_rate,ratestep,period,periodstep):
    """
    This function generates the payment matrix at various rates and terms
    """
    # Generates a matrix of 6 x 9 matrix with payments at various 
    # Declaring Row iterator
    termrows = [i for i in range(5,45,periodstep)]

    # Declaring Column Iterator
    #ratecolumn = [(round(j*100,3)) for j in np.arange(initial_rate,initial_rate+ 4.5/100,ratestep)]
    ratecolumn = [(round(j*100,3)) for j in np.arange(initial_rate - 2/100,initial_rate,ratestep)]  +\
    [(round(j*100,3)) for j in np.arange(initial_rate+ratestep,initial_rate+ 2.25/100,ratestep)] 
    
    # Naming axis and index
    mymatrix = pd.DataFrame(columns =ratecolumn ,index=termrows).rename_axis('Annual rate',axis=1)    
    mymatrix['Period']=termrows
    mymatrix.set_index('Period',inplace=True)
    
    # Generating Payment matrix for generated list of period and rates
    for i in termrows:
        for j in ratecolumn:
            mymatrix.at[i,j] = -npf.pmt(float(j/100)/paymentfreq,i*paymentfreq,principal)
            
    return (mymatrix)

In [5]:
def generate_payment_schedule(principal,annual_interest_rate,per,nper):
    """
    This function generates the amortization schedule
    """
    # Declaration of Variables
    periodic_interest_rate = annual_interest_rate/nper
    no_of_payments = nper*per
    
    # Defining the Structure of DataFrame
    columnnames =['Period','Opening Balance','Payment','Interest Expense','Repayment of Principal','Closing Balance']

    # Filling Static Columns & Index
    period=[i for i in range(1,no_of_payments+1)]
    
    # Formatting the DataFrame
    # pd.options.display.float_format = '₦{:,.2f}'.format
 
    # Initialization of the DataFrame
    mymatrix = pd.DataFrame(columns =columnnames,index=period)
    
    # Calculations 
    mymatrix.at[1,'Opening Balance']=principal
    mymatrix['Period']=period
    mymatrix.set_index('Period',inplace=True)
    mymatrix['Payment']=-npf.pmt(periodic_interest_rate,no_of_payments,principal)
    mymatrix['Interest Expense']=-npf.ipmt(periodic_interest_rate,mymatrix.index, no_of_payments,principal)
    mymatrix['Repayment of Principal']= -npf.ppmt(periodic_interest_rate,mymatrix.index,no_of_payments,principal)
    

    #Calculation of dynamic part of Amortization Schedule
    for i in period:
        if i>1:
            mymatrix['Closing Balance']= mymatrix['Opening Balance']-mymatrix['Repayment of Principal']
            mymatrix.at[i,'Opening Balance']=mymatrix.at[i-1,'Closing Balance']
        if mymatrix.at[i,'Opening Balance']-mymatrix.at[i,'Repayment of Principal']<0.1:
            mymatrix.at[i,'Closing Balance']=0


    mymatrix.at[1,'Opening Balance']=principal
    return (mymatrix)

In [6]:
def generate_summary(principal,initial_rate,per,nper):
    """
    This function generates summary for amortization schedule
    """
    mysum = generate_payment_schedule(principal,initial_rate,per,nper)
    total_interest = mysum["Interest Expense"].sum().round(2)
    pmt = -npf.pmt(initial_rate/nper,per*nper,principal)
    total=total_interest+principal
    mymatrix = pd.DataFrame(['₦'+str(pmt.round(2)),'₦'+str(total_interest.round(2)),'₦'+str(total.round(2))],
                            columns=[""],
                            index=["Payment per period","Total Interest","Total Payments"])
    
    return mymatrix

In [7]:
def amortization_table():
    """ Calculate the loan amortization schedule given the loan details

     Arguments:
        annual_interest_rate: The annual interest rate for this loan
        per: Number of years for the loan
        nper: Number of payments in a year
        principal: Amount borrowed

    Returns:
        matrix : Returns a 6 x 9 matrix of payments at various rates and term
        schedule: Amortization schedule as a pandas dataframe
        summary: Pandas dataframe that summarizes the payoff information
    """
    # Taking User Input     
    principalval = InputValidation("Principal")
    principal= principalval.check_positive_integer()

    rateval = InputValidation("Annual Interest Rate")
    annual_interest_rate=rateval.check_positive_float()/100
    
    pervalidation = InputValidation("Loan Term in years")
    per = int(pervalidation.finiteterm_check())

    npervalidation = InputValidation("No of Payments per year")
    nper = int(npervalidation.finiteterm_check())
    
    # Input information on the loan
    loan_df = pd.DataFrame(['₦'+ str(principal),str(annual_interest_rate*100)+'%',per,nper],
                            columns=[""],
                            index=["Loan Amount","Annual Rate of Interest","Number of Years","Payments per Year"])
    

    # Payment at various rates vs term
    matrix = generate_matrix(principal,nper,annual_interest_rate,0.005,per,5)

    # Amortization Schedule
    schedule = generate_payment_schedule(principal,annual_interest_rate,per,nper)
    summary= generate_summary(principal,annual_interest_rate,per,nper)
    
    
    display(pd.concat([loan_df, summary], axis=0).style.set_caption("Loan Summary").set_table_styles([
                                      {'selector' : '',
                                       'props' : [('background-color','white'),
                                                  ('border','2px solid black')]},
                                      {'selector': 'caption',
                                       'props': [('color', '#4f4646'),
                                                 ('font-size', '16px'),
                                                 ('text-align', 'center')]}]))
    
    display(matrix.style.set_caption("Two-dimensional Sensitivity analysis on Payment per Period").set_table_styles([
                            {'selector' : '',
                            'props' : [('background-color','white'),
                                       ('border','2px solid black')]},
                            {'selector': 'caption',
                            'props': [('color', '#4f4646'),
                                      ('font-size', '16px'),
                                      ('text-align', 'center')]}]).format('${:,.2f}'))

    print("\n \033[1m Based on the information you entered, your payment is {} for {} years with a rate of {}%\033[1m".\
          format(summary.at["Payment per period",""],per,annual_interest_rate*100))
    
    display(schedule.style.set_caption("Payment Schedule").set_table_styles([
                            {'selector' : '',
                            'props' : [('background-color','white'),
                                       ('border','2px solid black')]},
                            {'selector': 'caption',
                            'props': [('color', '#4f4646'),
                                      ('font-size', '16px'),
                                      ('text-align', 'center')]}]).format('₦{:,.2f}'))
    

In [8]:
amortization_table()


Please Enter a positive integer value for Principal:5000000

Please Enter a positive integer value for Annual Interest Rate:5

Please Enter a positive integer value for Loan Term in years:10

Please Enter a positive integer value for No of Payments per year:12


,
Loan Amount,$5000000.0
Annual Rate of Interest,5.0%
Number of Years,10
Payments per Year,12
Payment per period,$53032.76
Total Interest,$1363930.91
Total Payments,$6363930.91


Annual rate,3.000000,3.500000,4.000000,4.500000,5.500000,6.000000,6.500000,7.000000
Period,,,,,,,,
5,"$89,843.45","$90,958.72","$92,082.61","$93,215.10","$95,505.81","$96,664.01","$97,830.74","$99,005.99"
10,"$48,280.37","$49,442.93","$50,622.57","$51,819.20","$54,263.14","$55,510.25","$56,773.99","$58,054.24"
15,"$34,529.08","$35,744.13","$36,984.40","$38,249.66","$40,854.17","$42,192.84","$43,555.37","$44,941.41"
20,"$27,729.88","$28,997.99","$30,299.02","$31,632.47","$34,394.37","$35,821.55","$37,278.66","$38,764.95"
25,"$23,710.57","$25,031.18","$26,391.84","$27,791.62","$30,704.37","$32,215.07","$33,760.36","$35,338.96"
30,"$21,080.20","$22,452.23","$23,870.76","$25,334.27","$28,389.45","$29,977.53","$31,603.40","$33,265.12"
35,"$19,242.51","$20,664.53","$22,138.74","$23,662.84","$26,850.81","$28,509.49","$30,207.72","$31,942.82"
40,"$17,899.22","$19,369.55","$20,896.92","$22,478.14","$25,788.51","$27,510.68","$29,272.84","$31,071.56"



  Based on the information you entered, your payment is $53032.76 for 10 years with a rate of 5.0%


,Opening Balance,Payment,Interest Expense,Repayment of Principal,Closing Balance
Period,,,,,
1,"$5,000,000.00","$53,032.76","$20,833.33","$32,199.42","$4,967,800.58"
2,"$4,967,800.58","$53,032.76","$20,699.17","$32,333.59","$4,935,466.99"
3,"$4,935,466.99","$53,032.76","$20,564.45","$32,468.31","$4,902,998.68"
4,"$4,902,998.68","$53,032.76","$20,429.16","$32,603.60","$4,870,395.08"
5,"$4,870,395.08","$53,032.76","$20,293.31","$32,739.44","$4,837,655.63"
6,"$4,837,655.63","$53,032.76","$20,156.90","$32,875.86","$4,804,779.77"
7,"$4,804,779.77","$53,032.76","$20,019.92","$33,012.84","$4,771,766.93"
8,"$4,771,766.93","$53,032.76","$19,882.36","$33,150.40","$4,738,616.54"
9,"$4,738,616.54","$53,032.76","$19,744.24","$33,288.52","$4,705,328.02"
